In [3]:
import os
from osgeo import gdal
import numpy as np

# Input GeoTIFF file
input_geotiff = "hervanta_iso.tiff"

# Output directory for patches
output_dir = "patches"
os.makedirs(output_dir, exist_ok=True)

# Patch size (in pixels)
patch_size_x = 1024  # Width of the patch
patch_size_y = 1024 # Height of the patch

# Open the GeoTIFF file
ds = gdal.Open(input_geotiff)

if ds is None:
    print("Failed to open the GeoTIFF file.")
    exit(1)

# Get image dimensions
width = ds.RasterXSize
height = ds.RasterYSize

# Loop through the image and extract patches
for x in range(0, width, patch_size_x):
    for y in range(0, height, patch_size_y):
        x_offset = x
        y_offset = y
        x_size = min(patch_size_x, width - x_offset)
        y_size = min(patch_size_y, height - y_offset)

        # Read the data for the patch
        data = ds.ReadAsArray(x_offset, y_offset, x_size, y_size)

        # Save the patch as a new GeoTIFF file
        patch_filename = os.path.join(output_dir, f"patch_{x}_{y}.tif")
        driver = gdal.GetDriverByName("GTiff")
        patch_ds = driver.Create(patch_filename, x_size, y_size, ds.RasterCount, ds.GetRasterBand(1).DataType)
        patch_ds.SetProjection(ds.GetProjection())
        patch_ds.SetGeoTransform((ds.GetGeoTransform()[0] + x_offset * ds.GetGeoTransform()[1],
                                  ds.GetGeoTransform()[1],
                                  ds.GetGeoTransform()[2],
                                  ds.GetGeoTransform()[3] + y_offset * ds.GetGeoTransform()[5],
                                  ds.GetGeoTransform()[4],
                                  ds.GetGeoTransform()[5]))
        for band_num in range(1, ds.RasterCount + 1):
            band = ds.GetRasterBand(band_num)
            patch_band = patch_ds.GetRasterBand(band_num)
            patch_band.WriteArray(data[band_num - 1, :, :])
            patch_band.FlushCache()

        patch_ds = None

# Close the input GeoTIFF dataset
ds = None

print("Patching complete.")

Patching complete.
